<a href="https://colab.research.google.com/github/sumaaithal/PySpark_30Days_Challenge/blob/main/pyspark26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [645 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,617 kB]
Get:13 http://archive.ubuntu.com/ubuntu j

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [4]:
data = [("James","M",60000),("Michael","M",70000),
        ("Robert",None,400000),("Maria","F",500000),
        ("Jen","",None)]
columns = ["name","gender","salary"]

In [6]:
df = spark.createDataFrame(data=data,schema=columns)
df.show()

+-------+------+------+
|   name|gender|salary|
+-------+------+------+
|  James|     M| 60000|
|Michael|     M| 70000|
| Robert|  NULL|400000|
|  Maria|     F|500000|
|    Jen|      |  NULL|
+-------+------+------+



In [21]:
from pyspark.sql.functions import when,col,expr

In [14]:
df2 = df.withColumn("new_gender",
                    when(df.gender=='M',"Male")
                    .when(df.gender== 'F',"Female")
                    .when(df.gender.isNull(),"")
                    .otherwise(col("gender")) )

In [15]:
df2.show()

+-------+------+------+----------+
|   name|gender|salary|new_gender|
+-------+------+------+----------+
|  James|     M| 60000|      Male|
|Michael|     M| 70000|      Male|
| Robert|  NULL|400000|          |
|  Maria|     F|500000|    Female|
|    Jen|      |  NULL|          |
+-------+------+------+----------+



In [19]:
df3 = df.select(col("*"),when(df.gender=='M',"Male")
                    .when(df.gender== 'F',"Female")
                    .when(df.gender.isNull(),"")
                    .otherwise(col("gender")).alias("new_gender"))

In [20]:
df3.show()

+-------+------+------+----------+
|   name|gender|salary|new_gender|
+-------+------+------+----------+
|  James|     M| 60000|      Male|
|Michael|     M| 70000|      Male|
| Robert|  NULL|400000|          |
|  Maria|     F|500000|    Female|
|    Jen|      |  NULL|          |
+-------+------+------+----------+



In [22]:
df4 = df.withColumn("new_gender",
                    expr("CASE WHEN gender == 'M' THEN 'Male' "+
                         "WHEN gender == 'F' THEN 'Female' "+
                         "WHEN gender == NULL THEN ''"+
                         "ELSE gender END"))

In [23]:
df4.show()

+-------+------+------+----------+
|   name|gender|salary|new_gender|
+-------+------+------+----------+
|  James|     M| 60000|      Male|
|Michael|     M| 70000|      Male|
| Robert|  NULL|400000|      NULL|
|  Maria|     F|500000|    Female|
|    Jen|      |  NULL|          |
+-------+------+------+----------+



In [28]:
df5 = df.select("*",
                expr("CASE WHEN gender == 'M' THEN 'Male' "+
                         "WHEN gender == 'F' THEN 'Female' "+
                         "WHEN gender IS NULL THEN ' '"+
                         "ELSE gender END").alias("new_gender"))

In [29]:
df5.show()

+-------+------+------+----------+
|   name|gender|salary|new_gender|
+-------+------+------+----------+
|  James|     M| 60000|      Male|
|Michael|     M| 70000|      Male|
| Robert|  NULL|400000|          |
|  Maria|     F|500000|    Female|
|    Jen|      |  NULL|          |
+-------+------+------+----------+



In [31]:
df.createOrReplaceTempView("EMP")

In [36]:
spark.sql("SELECT name, CASE WHEN gender = 'M' THEN 'Male'"+
          "WHEN gender = 'F' THEN 'Female'"+
          "WHEN gender IS NULL THEN ''"+
           "ELSE gender END AS new_gender FROM EMP").show()

+-------+----------+
|   name|new_gender|
+-------+----------+
|  James|      Male|
|Michael|      Male|
| Robert|          |
|  Maria|    Female|
|    Jen|          |
+-------+----------+

